In [1]:
from io import StringIO
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utils

In [2]:
lines = '\n'.join([open(path, 'r').read() for path in Path('../results/mc').glob('*')])
df = pd.read_csv(StringIO(lines), sep=';', header=None)
df.columns = utils.results_columns
df['case'] = df['case'].apply(utils.format_case_name)
# df = df.loc[df['n_buses'].isin([73, 118, 179, 300, 500, 793, 1354, 1888])]
# df = df.loc[df['algorithm'].isin(['1ST', '2ST', 'ws'])]

In [3]:
mc = df.groupby(['case', 'n_buses', 'n_clusters', 'algorithm'])['maximum_congestion', 'runtime'].\
    sum().\
    unstack().\
    sort_values(['n_buses', 'n_clusters']).\
    droplevel('n_buses').\
    round(2)

/var/folders/wh/bdq353fj58j61dwg1ksz7br40000gn/T/ipykernel_87584/2587953096.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mc = df.groupby(['case', 'n_buses', 'n_clusters', 'algorithm'])['maximum_congestion', 'runtime'].\


In [4]:
mc

maximum_congestion             runtime                
algorithm                             1ST   2ST    ws     1ST    2ST       ws
case        n_clusters                                                       
IEEE-300    2                         NaN  1.16  1.06     NaN   1.01   601.26
            3                         NaN  1.16  1.09     NaN   1.26   601.46
            4                         NaN  1.38  1.09     NaN   1.64   601.80
            5                         NaN  1.63  1.09     NaN   2.16   602.07
GOC-500     2                         NaN  1.00  1.00     NaN   1.35     2.11
            3                         NaN  1.00  1.00     NaN   1.74     2.52
            4                         NaN  1.00  1.00     NaN   2.34     3.59
            5                         NaN  1.00  1.00     NaN   2.84     4.91
SDET-588    2                        1.02  1.44  1.02  204.12   1.53   189.27
            3                         NaN  1.32  1.00     NaN   2.12   256.37
            4                         NaN  1.29  1.00     NaN   2.56   182.36
            5                         NaN  1.26  1.00     NaN   3.15    19.10
GOC-793     2                         NaN  2.06  1.05     NaN   2.22   602.90
            3                         NaN  1.57  1.00     NaN   3.08   603.55
            4                         NaN  1.57  1.01     NaN   4.93   605.17
            5                         NaN  1.64  1.07     NaN  14.77   606.19
PEGASE-1354 2                         NaN  1.56  1.11     NaN   4.21   606.11
            3                         NaN  1.55  1.13     NaN   5.79   611.29
            4                         NaN  1.55  1.44     NaN   8.53   612.61
            5                         NaN  1.55  1.20     NaN  10.34   614.77
RTE-1888    2                         NaN  1.00  1.00     NaN   5.49     9.56
            3                         NaN  1.00  1.00     NaN   6.87    11.18
            4                         NaN  1.00  1.00     NaN  23.40    12.77
            5                         NaN  1.00  1.00     NaN  34.96    21.15
GOC-2000    2                         NaN  1.13  1.13     NaN   8.19   609.62
            3                         NaN  1.13  1.13     NaN  14.33  1210.97
            4                         NaN  1.08   NaN     NaN  22.10      NaN
            5                         NaN  1.77   NaN     NaN  53.27      NaN
RTE-2848    2                         NaN  1.00  1.00     NaN   8.77    15.84
            3                         NaN  1.00  1.00     NaN  11.43    31.00
            4                         NaN  1.00  1.00     NaN  13.36    48.46
            5                         NaN  1.00  1.00     NaN  17.89   144.15

In [5]:
mc['maximum_congestion']
single = mc[('maximum_congestion', '1ST')] 
ws = mc[('maximum_congestion', 'ws')]
merged_obj = single.where(~single.isna(), ws)

# Compute runtimes
single = mc[('runtime', '1ST')] 
ws = mc[('runtime', 'ws')]
merged_runtime = single.where(~single.isna(), ws)
merged_runtime = np.minimum(merged_runtime, 600)


In [6]:
new = mc
new = new.drop([('maximum_congestion', '1ST'), ('maximum_congestion', 'ws'), ('runtime', '1ST'), ('runtime', 'ws')], axis=1)

In [7]:
new[('maximum_congestion', '1ST')] = merged_obj
new[('runtime', '1ST')] = merged_runtime

two = new[('maximum_congestion', '2ST')]
new[('maximum_congestion', 'gap')] = round((two - merged_obj) / merged_obj * 100, 2)
print(new.sort_index(axis=1).to_latex())

\begin{tabular}{llrrrrr}
\toprule
         & {} & \multicolumn{3}{l}{maximum\_congestion} & \multicolumn{2}{l}{runtime} \\
         & algorithm &                1ST &   2ST &    gap &     1ST &    2ST \\
case & n\_clusters &                    &       &        &         &        \\
\midrule
IEEE-300 & 2 &               1.06 &  1.16 &   9.43 &  600.00 &   1.01 \\
         & 3 &               1.09 &  1.16 &   6.42 &  600.00 &   1.26 \\
         & 4 &               1.09 &  1.38 &  26.61 &  600.00 &   1.64 \\
         & 5 &               1.09 &  1.63 &  49.54 &  600.00 &   2.16 \\
GOC-500 & 2 &               1.00 &  1.00 &   0.00 &    2.11 &   1.35 \\
         & 3 &               1.00 &  1.00 &   0.00 &    2.52 &   1.74 \\
         & 4 &               1.00 &  1.00 &   0.00 &    3.59 &   2.34 \\
         & 5 &               1.00 &  1.00 &   0.00 &    4.91 &   2.84 \\
SDET-588 & 2 &               1.02 &  1.44 &  41.18 &  204.12 &   1.53 \\
         & 3 &               1.00 &  1.32 &  32.00 &

/var/folders/wh/bdq353fj58j61dwg1ksz7br40000gn/T/ipykernel_87584/1491543589.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(new.sort_index(axis=1).to_latex())


In [8]:
print(new.to_latex())

\begin{tabular}{llrrrrr}
\toprule
         & {} & maximum\_congestion & runtime & maximum\_congestion & runtime & maximum\_congestion \\
         & algorithm &                2ST &     2ST &                1ST &     1ST &                gap \\
case & n\_clusters &                    &         &                    &         &                    \\
\midrule
IEEE-300 & 2 &               1.16 &    1.01 &               1.06 &  600.00 &               9.43 \\
         & 3 &               1.16 &    1.26 &               1.09 &  600.00 &               6.42 \\
         & 4 &               1.38 &    1.64 &               1.09 &  600.00 &              26.61 \\
         & 5 &               1.63 &    2.16 &               1.09 &  600.00 &              49.54 \\
GOC-500 & 2 &               1.00 &    1.35 &               1.00 &    2.11 &               0.00 \\
         & 3 &               1.00 &    1.74 &               1.00 &    2.52 &               0.00 \\
         & 4 &               1.00 &    2.34 &    

/var/folders/wh/bdq353fj58j61dwg1ksz7br40000gn/T/ipykernel_87584/2372328195.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(new.to_latex())
